In [1]:
import csv
import itertools
import pandas as pd
import numpy as np
from math import log, e

In [2]:
# Connection table between nodes
connections = pd.read_csv("Brain1Connections.csv", header=None)

In [3]:
# Positions of nodes (X,Y,Z)
positions = pd.read_csv("Brain1Positions.csv", header=None, sep=";")
positions[0][0] = positions[0][0][:-1]

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Convert data to numerical value
positions[0] = positions[0].astype(float)

In [5]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [6]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [7]:
# Calculate distance between 2 points in Euclidean space

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [8]:
Euclidean_dist(0,8)

60.821899890312004

In [9]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p):
    neighbours = []
    
    for i in range(83):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [11]:
# Add a key-value pair to the existing dictionary
def add_element(dict, key, value):
    if key not in dict:
        dict[key] = []
    dict[key].append(value)

In [12]:
# Returns with a dictionary. Key is the number of the node and 
# the value is the distant between the current node(act) and its neighbour.
# The list is ascendant by the distance.
def get_dist(act, neighbours):
    dist = {}
    
    for i in neighbours:
        add_element(dist, i, Euclidean_dist(act, i))
    
    dist = sorted(dist.items(), key=lambda x: x[1])
    
    return dist

In [13]:
dict = get_dist(9, n)
print(dict)
dict[5]

[(8, [19.814297429824563]), (5, [32.373310145716125]), (33, [38.951915535245845]), (35, [42.706932767219335]), (36, [43.43564309241848]), (34, [44.451031502115114]), (7, [44.84036410692491]), (37, [45.1517229111004]), (30, [49.170642203670006]), (31, [49.321624779058034]), (4, [53.83213143549891]), (17, [54.190515949666526]), (18, [55.08246698267312]), (6, [57.51950554766205]), (38, [57.63795524503112]), (29, [58.665717994905336]), (40, [59.13731145521251]), (28, [65.19262363076962]), (24, [67.90906887217683]), (1, [68.12905048052522]), (82, [69.84525602598247]), (27, [72.59785793831435]), (3, [77.74387478985666]), (23, [77.83006533931774]), (22, [82.94140517826806]), (2, [89.43970444117359])]


(34, [44.451031502115114])

In [14]:
# Calculates the full lenght of the path through the nodes.
def path_dist(g):
    dist = 0
    
    for i in range(len(g)-1):
        dist = dist + Euclidean_dist(g[i], g[i+1])
    
    return dist

In [15]:
path_dist([0, 17, 10])

146.86216180971223

In [16]:
[0, 17, 10]
[110.26496976737384, 36.59719204233838]

[110.26496976737384, 36.59719204233838]

In [17]:
# Entropy by numpy library
def entropy(labels, base=None):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    value, counts = np.unique(labels, return_counts=True)
    probs = counts/n_labels
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.
    
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)
        
    return ent

In [18]:
entropy([0, 17, 10])

1.0986122886681096

# Algorithm 1.

In [19]:
# The algorithm chooses the node which one is the closest to the current node and they are neighbours.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

'''def greedy1(p, z):
    ready = False
    routing = []
    routing.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing.append(z)
            return routing
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            act = neighbours[0]
            routing.append(act)
            
        elif len(neighbours) > 1:
            dist = get_dist(act, neighbours)
            
            # print("Neighbours: ", neighbours)
            # print("DIST: ", dist)
            # print()
            
            not_found = False
            for i in dist:
                if i[0] not in routing:
                    act = i[0]
                    routing.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                act = dist[0][0]
                routing.append(act)'''

'def greedy1(p, z):\n    ready = False\n    routing = []\n    routing.append(p)\n    act = p       # current node\n    \n    while ready==False:\n        neighbours = get_neighbours(act)\n        \n        # if the \'z\' is connected with \'act\'\n        if z in neighbours:\n            routing.append(z)\n            return routing\n        \n        # if \'act\' has no neighbours \n        elif len(neighbours)==0:\n            return "No neighbours"\n        \n        # if \'act\' has just 1 neighbour\n        elif len(neighbours) == 1:\n            act = neighbours[0]\n            routing.append(act)\n            \n        elif len(neighbours) > 1:\n            dist = get_dist(act, neighbours)\n            \n            # print("Neighbours: ", neighbours)\n            # print("DIST: ", dist)\n            # print()\n            \n            not_found = False\n            for i in dist:\n                if i[0] not in routing:\n                    act = i[0]\n                    rout

In [20]:
'''routing_map_1 = []

# Add the path of p->z to the routing table 
def greedy1_algorithm(p,z):
    path = greedy1(p,z)
    if path not in routing_map_1:
        routing_map_1.append(path)'''

'routing_map_1 = []\n\n# Add the path of p->z to the routing table \ndef greedy1_algorithm(p,z):\n    path = greedy1(p,z)\n    if path not in routing_map_1:\n        routing_map_1.append(path)'

In [21]:
'''# Iterate throught all of the nodes and run the greedy1 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy1_algorithm(i,j)'''

'# Iterate throught all of the nodes and run the greedy1 algorithm on them.\nfor i in range(83):\n    for j in range(83):\n        if i!= j:\n            greedy1_algorithm(i,j)'

In [22]:
'''# Calculate the average of the nodes of the paths
sum_nodes1 = 0
for i in range(6806):
    sum_nodes1 = sum_nodes1 + len(routing_map_1[i])
    
avr_nodes1 = sum_nodes1 / 6806
avr_nodes1'''

'# Calculate the average of the nodes of the paths\nsum_nodes1 = 0\nfor i in range(6806):\n    sum_nodes1 = sum_nodes1 + len(routing_map_1[i])\n    \navr_nodes1 = sum_nodes1 / 6806\navr_nodes1'

In [23]:
'''# Calculate the average distance of the paths
sum_dist1 = 0
for i in range(6806):
    sum_dist1 = sum_dist1 + path_dist(routing_map_1[i])
    
avr_dist1 = sum_dist1 / 6806
avr_dist1'''

'# Calculate the average distance of the paths\nsum_dist1 = 0\nfor i in range(6806):\n    sum_dist1 = sum_dist1 + path_dist(routing_map_1[i])\n    \navr_dist1 = sum_dist1 / 6806\navr_dist1'

# Algorithm 2.

In [24]:
# The algorithm chooses the neighbour node, which one is the closest the to end node.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy2(p, z, routing_nodes, routing_dist):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours) == 0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            not_found = False
            for i in dist:
                if i[0] not in routing_nodes:
                    routing_dist.append(Euclidean_dist(act, i[0]))
                    act = i[0]
                    routing_nodes.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                routing_dist.append(Euclidean_dist(act, dist[0][0]))
                act = dist[0][0]
                routing_nodes.append(act)

In [25]:
routing_nodes = []
routing_dist = []

greedy2(0,10,routing_nodes,routing_dist)
print(routing_nodes)
print(routing_dist)

[0, 17, 10]
[110.26496976737384, 36.59719204233838]


In [26]:
routing_map_nodes2 = []
routing_map_dist2 = []

# Add the path of p->z to the routing table 
def greedy2_algorithm(p,z):
    routing_nodes = []
    routing_dist = []
    greedy2(p,z,routing_nodes,routing_dist)
    if routing_nodes not in routing_map_nodes2:
        routing_map_nodes2.append(routing_nodes)
        routing_map_dist2.append(routing_dist)

In [27]:
# Iterate throught all of the nodes and run the greedy2 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy2_algorithm(i,j)

In [43]:
print(len(routing_map_nodes2))

# Calculate the average of the nodes of the paths
sum_nodes2 = 0
for i in range(6806):
    sum_nodes2 = sum_nodes2 + len(routing_map_nodes2[i])
    
avr_nodes2 = sum_nodes2 / 6806
avr_nodes2

6806


3.0113135468704084

In [29]:
# Calculate the average distance of the paths
sum_dist2 = 0
for i in range(6806):
    sum_dist2 = sum_dist2 + path_dist(routing_map_nodes2[i])
    
avr_dist2 = sum_dist2 / 6806
avr_dist2

84.29096240285445

In [51]:
# Calculate entropy for each path

nodes_entropy2 = []
dist_entropy2 = []

for i in range(len(routing_map_nodes2)):
    nodes_entropy2.append(entropy(routing_map_nodes2[i]))
    dist_entropy2.append(entropy(routing_map_dist2[i]))

In [68]:
# Average entropy

sum_ent2 = 0
for i in range(6806):
    # sum_ent2 += dist_entropy2[i]
    sum_ent2 += nodes_entropy2[i]
    
avr_ent2 = sum_ent2 / 6806
avr_ent2

1.0622088782157757

# Algorithm 3.

In [33]:
# The algorithm chooses the neighbour node, through the distance to the end node from 'p' is the shortest.
# If the closest node is already on the routing list, the algorithm will choose the next one in the ascendant list.
# A node could be on the routing table several times, if the algorithm already used all of the elements in the neighbour list.
# If that happens, it will choose the closest one again.

def greedy3(p, z, routing_nodes, routing_dist):
    ready = False
    routing_nodes.append(p)
    act = p       # current node
    
    while ready==False:
        neighbours = get_neighbours(act)
        
        # Calculates the distances of the neighbour nodes from the end node
        dist = get_dist(z, neighbours)
        
        # if the 'z' is connected with 'act'
        if z in neighbours:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            routing_nodes.append(z)
            ready=True
        
        # if 'act' has no neighbours 
        elif len(neighbours)==0:
            return "No neighbours"
        
        # if 'act' has just 1 neighbour
        elif len(neighbours) == 1:
            routing_dist.append(Euclidean_dist(act, dist[0][0]))
            act = neighbours[0]
            routing_nodes.append(act)
            
        elif len(neighbours) > 1:
            # distance from 'p' to the neighbours
            dist1 = get_dist(act, neighbours)
            # distance from the neighbours to the end node
            dist2 = get_dist(z, neighbours)
            dist = {}
            
            # Calculate the distance from 'p' through the neighbours to the end node
            for i in range(len(neighbours)):
                for j in range(len(neighbours)):
                    if dist1[i][0]==dist2[j][0]:
                        val1 = dist1[i][1]
                        val2 = dist2[j][1]
                        add_element(dist, dist1[i][0], val1[0]+val2[0])
            dist = sorted(dist.items(), key=lambda x: x[1])
            
            not_found = False
            for i in dist:
                if i[0] not in routing_nodes:
                    routing_dist.append(Euclidean_dist(act, i[0]))
                    act = i[0]
                    routing_nodes.append(act)
                    not_found = True
                    break
            # If there are no more choosable node on the list     
            if not_found == False:
                routing_dist.append(Euclidean_dist(act, dist[0][0]))
                act = dist[0][0]
                routing_nodes.append(act)

In [34]:
routing_nodes2 = []
routing_dist2 = []

greedy3(0,10,routing_nodes2,routing_dist2)
print(routing_nodes2)
print(routing_dist2)

[0, 35, 10]
[35.057875034510346, 53.27508125272272]


In [44]:
routing_map_nodes3 = []
routing_map_dist3 = []

# Add the path of p->z to the routing table 
def greedy3_algorithm(p,z):
    routing_nodes = []
    routing_dist = []
    greedy3(p,z,routing_nodes,routing_dist)
    if routing_nodes not in routing_map_nodes3:
        routing_map_nodes3.append(routing_nodes)
        routing_map_dist3.append(routing_dist)

In [45]:
# Iterate throught all of the nodes and run the greedy3 algorithm on them.
for i in range(83):
    for j in range(83):
        if i!= j:
            greedy3_algorithm(i,j)
            
len(routing_map_nodes3)

6806

In [46]:
# Calculate the average of the nodes of the paths
sum_nodes3 = 0
for i in range(6806):
    sum_nodes3 = sum_nodes3 + len(routing_map_nodes3[i])
    
avr_nodes3 = sum_nodes3 / 6806
avr_nodes3

3.564942697619747

In [50]:
# Calculate the average distance of the paths
sum_dist3 = 0
for i in range(6806):
    sum_dist3 = sum_dist3 + path_dist(routing_map_nodes3[i])
    
avr_dist3 = sum_dist3 / 6806
avr_dist3

78.99626993739102

In [52]:
# Calculate entropy for each path

nodes_entropy3 = []
dist_entropy3 = []

for i in range(len(routing_map_nodes3)):
    nodes_entropy3.append(entropy(routing_map_nodes3[i]))
    dist_entropy3.append(entropy(routing_map_dist3[i]))

In [67]:
# Average entropy

sum_ent3 = 0
for i in range(6806):
    #sum_ent3 += dist_entropy3[i]
    sum_ent3 += nodes_entropy3[i]
    
avr_ent3 = sum_ent3 / 6806
avr_ent3

1.1864648672380205

## Algorithm 1:
    - Average node number of paths:
        8.070232148104614
    - Average path lenght:
        162.67517508376523
    - Average entropy of the distance: 
        0.6037558913009416
    
## Algorithm 2:
    - Average node number of paths:
        3.0113135468704084
    - Average path lenght:
        84.29096240285445
    
## Algorithm 3:
    - Average node number of paths: 
        3.564942697619747 
    - Average path lenght: 
        78.99626993739102
    - Average entropy of the distance:
        0.7678959699816416

In [55]:
'''with open('alg2nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_nodes2[index]] for index in range(0, len(routing_map_nodes2)))
    
with open('alg2dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_dist2[index]] for index in range(0, len(routing_map_dist2)))
    
with open('alg3nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_nodes3[index]] for index in range(0, len(routing_map_nodes3)))
    
with open('alg3dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([routing_map_dist3[index]] for index in range(0, len(routing_map_dist3)))
    
with open('ent2nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([nodes_entropy2[index]] for index in range(0, len(nodes_entropy2)))
    
with open('ent2dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([dist_entropy2[index]] for index in range(0, len(dist_entropy2)))
    
with open('ent3nodes.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([nodes_entropy3[index]] for index in range(0, len(nodes_entropy3)))
    
with open('ent3dist.csv', 'w', newline='') as csv_1:
  csv_out = csv.writer(csv_1)
  csv_out.writerows([dist_entropy3[index]] for index in range(0, len(dist_entropy3)))'''